In [19]:
# imports
import numpy as np
from matplotlib import pyplot as plt
from scipy import signal
import pandas as pd
from glob import glob as glob
import os
from neo import io
import seaborn as sb
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
plt.rcParams["font.size"]= 12
plt.rcParams["font.family"] = "Arial"

# functions for PSTH
from scipy.signal import gaussian
from scipy.ndimage import filters
def rate_binning(spike_times,time_bins,binsize):
    average = np.zeros((len(spike_times),len(time_bins)))
    for i,t in enumerate(time_bins):
        
        for chan in range(len(spike_times)):
            include = (spike_times[chan]>=t) & (spike_times[chan]<(t+binsize))
            average[chan,i] = sum(include)/binsize
    return average

def smoothing(signal,sd,binsize):
    ''' aplly gaussian filter per trial'''
    kernel = gaussian(signal.shape[-1],sd/binsize)
    ga = np.zeros(signal.shape)
    if len(signal.shape)>3:
        for dim1 in range(signal.shape[0]):
            for dim2 in range(signal.shape[1]):
                for dim3 in range(signal.shape[2]):
                    ga[dim1,dim2,dim3,:] = filters.convolve1d(signal[dim1,dim2,dim3,:], kernel/kernel.sum())
    else:
        for dim1 in range(signal.shape[0]):
            for dim2 in range(signal.shape[1]):
                ga[dim1,dim2,:] = filters.convolve1d(signal[dim1,dim2,:], kernel/kernel.sum())
                
    return ga

Load preprocessed data and extract sequence variables

In [20]:
# parameters
animal_id = 'A11' # A11, H07
exp = 'classical' # 'wang or classical
out_path = '/Volumes/Bellet/Local_Global/processedData/%s/'%animal_id # path to the data

# load preprocessed data
df = pd.read_pickle(os.path.join(out_path,'%s_stims_spikes_dataframe.pkl'%(animal_id)))

In [21]:
# extract basic variables
dates = np.unique(df.date)
ndates = len(dates)
nitems = np.max(df.ItemID)
stimID = np.unique(df.StimID)
nstim = len(stimID)
print(list(df.keys()))
print('Recording dates:',dates)

['PFC_MU', 'PPC_MU', 'PFC_SU', 'PPC_SU', 'TrialID', 'ItemID', 'StimID', 'StimName', 'StimOn', 'blockID', 'blockType', 'date', 'StimDur', 'ISIDur', 'RewardOn']
Recording dates: ['20200226' '20200228' '20200305' '20200306' '20200311' '20200312']


compute PSTH from spike times; only last stimulus

In [22]:
# compute psth of every stim presented

sf = 30000; # sampling frequency
tmin = -0.3 # sec, before each stim
tmax = 1.6 # sec, after each stim
nch = len(df.PFC_MU[0])

# create PSTH
binsize = 0.025 # sec
stepsize = 0.025 # sec, if stepsize=binsize: no overlap of bins
sd = 0.05 # standard deviation of gaussian kernel
time_bins = np.arange(tmin,tmax,stepsize)
time_bins_gauss = time_bins[int(4*sd/binsize):-int(4*sd/binsize)]
nbins = len(time_bins)

for date in dates:

    np.save(os.path.join(out_path,'time_bins_%s.npy'%date),time_bins)
    np.save(os.path.join(out_path,'time_bins_gauss_%s.npy'%date),time_bins_gauss)
    
    # get conditions
    last_item = np.where((df.ItemID==3) & (df.date==date))[0]
    Ntrials = len(last_item)

    # bin spikes from all channels and trials
    
    #######################
    # PFC
    print('Day:',date,' PFC')
    Rb_last = np.zeros((Ntrials,nch,nbins))
    for tr in range(Ntrials):
        ind = last_item[tr]
        Rb_last[tr,:] = rate_binning(df['PFC_MU'].iloc[ind],time_bins,binsize)
    np.save(os.path.join(out_path,'Rb_last_%s.npy'%date),Rb_last)
    
    # gaussian smoothing
    Rb_last = smoothing(Rb_last,sd,binsize)[:,:,int(4*sd/binsize):-int(4*sd/binsize)] # cut borders: 4*sd
    np.save(os.path.join(out_path,'Rb_last_gauss_%s.npy'%date),Rb_last)
    

Day: 20200226  PFC
Day: 20200228  PFC
Day: 20200305  PFC
Day: 20200306  PFC
Day: 20200311  PFC
Day: 20200312  PFC


In [ ]:
# GET PSTH FROM whole sequence, 
# consider entire trials only
sf = 30000; # sampling frequency
tmin = -0.3 # sec, before sequence start
tmax = 1.6 # sec, after last stim
nch = len(df.PFC_MU[0])

# create PSTH
binsize = 0.025 # sec
stepsize = 0.025 # sec, if stepsize=binsize: no overlap of bins
sd = 0.05 # standard deviation of gaussian kernel


for date in dates:

    print('Day:',date)
    if ~os.path.exists(os.path.join(out_path,'Rb_seq_%s.npy'%date)):
        # get conditions
        last_item = np.where((df.ItemID==3) & (df.date==date))[0]
        Ntrials = len(last_item)
    
        soa = (int(df.StimDur.iloc[last_item[0]]) + int(df.ISIDur.iloc[last_item[0]]))/1000.0
        time_bins = np.arange(tmin,3*soa+tmax,stepsize) # time bins for entire sequence
        time_bins_gauss = time_bins[int(4*sd/binsize):-int(4*sd/binsize)]
        nbins = len(time_bins)
        
        # bin spikes from all channels and trials
        Rb = np.zeros((Ntrials,nch,nbins))
        
        for tr in range(Ntrials):
            t = 0
            for i in range(4): # go until 4 stim backwards to start with 1st
                ind = last_item[tr]-3+i
                if i==3:
                    time_bins_single = np.arange(0,tmax,stepsize) # time bins around last stimulus
                elif i==0:
                    time_bins_single = np.arange(tmin,soa,stepsize) # time bins around 1st stimulus
                else:
                    time_bins_single = np.arange(0,soa,stepsize) # time bins around 2nd and 3rd stimulus
                Rb[tr,:,t:t+len(time_bins_single)] = rate_binning(df['PFC_MU'].iloc[ind],time_bins_single,binsize)
                t += len(time_bins_single)
        np.save(os.path.join(out_path,'Rb_seq_%s.npy'%date),Rb)
        np.save(os.path.join(out_path,'time_bins_seq_%s.npy'%date),time_bins)
        
        # gaussian smoothing
        Rb = smoothing(Rb,sd,binsize)[:,:,int(4*sd/binsize):-int(4*sd/binsize)] # cut borders: 4*sd
        np.save(os.path.join(out_path,'Rb_seq_gauss_%s.npy'%date),Rb)
        np.save(os.path.join(out_path,'time_bins_seq_gauss_%s.npy'%date),time_bins_gauss)
        